#**Parte 1: Caracterização e Análise de Dados**

**Grupo 10:**

Bruno Buti Ferreira Guilherme

João Vitor Vieira Neves

Júlia Paes de Viterbo - 2021032137

Lucas Affonso Pires

Lucas de Oliveira Ferreira

##**Preparação**

In [1]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [2]:
path = ''

def iam(nome):
    global path

    if nome == 'julia':
        path = '/content/drive/MyDrive/Trabalhos UFMG/FUTEBOL'
    elif nome == 'lucas a':
        path = '/caminho/do/lucas_a'
    elif nome == 'lucas o':
        path = '/caminho/do/lucas_o'
    elif nome == 'joao':
        path = '/caminho/do/joao'
    elif nome == 'bruno':
        path = '/caminho/do/bruno'
    else:
        path = '.'

In [3]:
#MODIFICAR!
iam('julia')
%cd $path

/content/drive/MyDrive/Trabalhos UFMG/FUTEBOL


In [4]:
import json
import bz2
import pandas as pd
import os
import pyarrow as pa
import pyarrow.parquet as pq
import psutil
from tqdm.notebook import tqdm
import time

###**Organização das tabelas**

In [5]:
#selecionando o melhor time da temporada pra pegar os jogos dele para processar (são muitos)
df = pd.read_csv('./PL_23_24/metadata.csv')
metadata_df = df
df.head(2)

,awayTeam,awayTeamKit,competition,date,endPeriod1,endPeriod2,homeTeam,homeTeamKit,homeTeamStartLeft,id,period1,period2,season,stadium,startPeriod1,startPeriod2,videos,week
0,"{'id': '11', 'name': 'Manchester City', 'short...","{'name': 'alt-1', 'primaryColor': '#FFFFFF', '...","{'id': '1', 'name': 'Premier League'}",2023-08-11T19:00:00,3157.691,6393.827,"{'id': '5', 'name': 'Burnley', 'shortName': 'B...","{'name': 'home', 'primaryColor': '#690F3C', 'p...",True,13335,50.06340,53.068067,2023-2024,"{'id': '6', 'name': 'Turf Moor', 'pitchLength'...",153.887,3209.743,"{'fps': 29.97, 'id': '11287', 'videoUrl': 'epi...",1
1,"{'id': '221', 'name': 'Nottingham Forest', 'sh...","{'name': '2023 Away', 'primaryColor': '#ffffff...","{'id': '1', 'name': 'Premier League'}",2023-08-12T11:30:00,3164.664,6375.008,"{'id': '2', 'name': 'Arsenal', 'shortName': 'A...","{'name': 'home', 'primaryColor': '#E10B17', 'p...",True,13336,50.98765,53.003550,2023-2024,"{'id': '3', 'name': 'Emirates Stadium', 'pitch...",105.405,3194.795,"{'fps': 29.97, 'id': '11300', 'videoUrl': 'epi...",1


In [6]:
#id corresponde aos arquivos [id].jsol.bz2 em ./PL_23_24
best_team = df['awayTeam'].iloc[0]
df = df[(df['awayTeam']==best_team) | (df['homeTeam']==best_team)]

df.head(2)

,awayTeam,awayTeamKit,competition,date,endPeriod1,endPeriod2,homeTeam,homeTeamKit,homeTeamStartLeft,id,period1,period2,season,stadium,startPeriod1,startPeriod2,videos,week
0,"{'id': '11', 'name': 'Manchester City', 'short...","{'name': 'alt-1', 'primaryColor': '#FFFFFF', '...","{'id': '1', 'name': 'Premier League'}",2023-08-11T19:00:00,3157.691,6393.827,"{'id': '5', 'name': 'Burnley', 'shortName': 'B...","{'name': 'home', 'primaryColor': '#690F3C', 'p...",True,13335,50.063400,53.068067,2023-2024,"{'id': '6', 'name': 'Turf Moor', 'pitchLength'...",153.887,3209.743,"{'fps': 29.97, 'id': '11287', 'videoUrl': 'epi...",1
15,"{'id': '13', 'name': 'Newcastle United', 'shor...","{'name': 'alt home', 'primaryColor': '#ffffff'...","{'id': '1', 'name': 'Premier League'}",2023-08-19T19:00:00,2909.009,5959.192,"{'id': '11', 'name': 'Manchester City', 'short...","{'name': 'home', 'primaryColor': '#6CABDD', 'p...",True,13350,48.076417,50.097867,2023-2024,"{'id': '12', 'name': 'Etihad Stadium', 'pitchL...",24.424,2953.320,"{'fps': 29.97, 'id': '11535', 'videoUrl': 'epi...",2


In [7]:
len(df['id'].unique())

28

In [8]:
ids = df['id'].unique().tolist()

In [9]:
processed_files = [] #INSERIR O QUE NÃO PRECISA
file_names = []
for i in ids:
  file_name = str(i) + '.jsonl.bz2'
  if file_name not in processed_files:
    file_names.append(file_name)
file_names

['13460.jsonl.bz2',
 '13471.jsonl.bz2',
 '13475.jsonl.bz2',
 '13490.jsonl.bz2',
 '13501.jsonl.bz2',
 '13521.jsonl.bz2',
 '13530.jsonl.bz2',
 '13543.jsonl.bz2',
 '13554.jsonl.bz2',
 '13557.jsonl.bz2',
 '13570.jsonl.bz2',
 '13580.jsonl.bz2',
 '13587.jsonl.bz2',
 '13600.jsonl.bz2',
 '13611.jsonl.bz2',
 '13510.jsonl.bz2']

In [11]:
file_names[14:]

['13611.jsonl.bz2', '13510.jsonl.bz2']

In [12]:
file_names = file_names[14:]

**NÃO RODAAAAAAAR**

In [13]:
#lista todos os .jsonl.bz2
jsonl_dir = './PL_23_24'
def get_all_filenames(jsonl_dir):
    return [f for f in os.listdir(jsonl_dir) if f in file_names]

def read_jsonl_bz2(filepath):
    data = []
    with bz2.open(filepath, 'rt') as file:
        for line in file:
            try:
                data.append(json.loads(line))
            except json.JSONDecodeError as e:
                print(f"Erro na linha: {line[:50]}... Erro: {e}")
    return data

#processamento dos dados de tracking
def flatten_tracking(df: pd.DataFrame) -> pd.DataFrame:
    columns_to_keep = ['gameRefId', 'period', 'periodGameClockTime', 'frameNum', 'game_event_id', 'possession_event_id']

    home = (
        df[columns_to_keep + ['homePlayersSmoothed']]
        .explode('homePlayersSmoothed')
        .dropna(subset=['homePlayersSmoothed'])
    ).reset_index(drop=True)
    home = pd.concat([
        home.drop(columns=['homePlayersSmoothed']),
        pd.json_normalize(home['homePlayersSmoothed'])
    ], axis=1)
    home['element'] = 'home'

    away = (
        df[columns_to_keep + ['awayPlayersSmoothed']]
        .explode('awayPlayersSmoothed')
        .dropna(subset=['awayPlayersSmoothed'])
    ).reset_index(drop=True)
    away = pd.concat([
        away.drop(columns=['awayPlayersSmoothed']),
        pd.json_normalize(away['awayPlayersSmoothed'])
    ], axis=1)
    away['element'] = 'away'

    balls = (
        df[columns_to_keep + ['ballsSmoothed']]
        .dropna(subset=['ballsSmoothed'])
    )
    balls = pd.concat([
        balls.drop(columns=['ballsSmoothed']),
        pd.json_normalize(balls['ballsSmoothed'])
    ], axis=1)
    balls['element']   = 'ball'
    balls['jerseyNum'] = None

    out = pd.concat([home, away, balls], ignore_index=True, sort=False).reset_index(drop=True)
    return out[columns_to_keep + ['element','jerseyNum','x','y']]

#pasta de destino a ser criada
parquet_dir = './Tracking Data'
os.makedirs(parquet_dir, exist_ok=True)

#conversão para .parquet
stored_tracking_data = get_all_filenames(jsonl_dir)

for file_name in tqdm(stored_tracking_data):
    print(f"Processando: {file_name}")

    json_path = os.path.join(jsonl_dir, file_name)
    json_data = read_jsonl_bz2(json_path)

    df = pd.DataFrame(json_data)
    flat_df = flatten_tracking(df)

    parquet_name = file_name.replace('.jsonl.bz2', '.parquet')
    parquet_path = os.path.join(parquet_dir, parquet_name)

    table = pa.Table.from_pandas(flat_df)
    pq.write_table(table, parquet_path)


  0%|          | 0/2 [00:00<?, ?it/s]

Processando: 13611.jsonl.bz2
Processando: 13510.jsonl.bz2


##**Leitura e análise iniciais**

In [ ]:
#concatenação dos jogos
rosters_df = pd.read_csv('./PL_23_24/rosters.csv')
players_df = pd.read_csv('./PL_23_24/players.csv')
df_335 = pd.read_parquet('./Tracking Data/13335.parquet')

##**Limpeza**

##**Caracterização final**